In [2]:
import requests
import sqlite3
import pandas as pd

# NOAA API token
api_token = 'rDZYpxCxcNrIUzeserZMcYDaaHCNPcFZ'

# connect to the DB
conn = sqlite3.connect('C:/Users/Mark Rozenberg/Foobal-Climate/Data/Main_DB.db')

### create table for locations id's

In [ ]:
headers = {
    'token': api_token
}

# Set up SQLite database
cursor = conn.cursor()

# Create table for locations
cursor.execute('''
CREATE TABLE IF NOT EXISTS locations (
    id TEXT PRIMARY KEY,
    name TEXT,
    mindate TEXT,
    maxdate TEXT,
    datacoverage REAL
)
''')

# Function to fetch locations from NOAA API
def fetch_locations(offset=1, limit=1000):
    url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/locations'
    params = {
        'locationcategoryid': 'CITY',
        'limit': limit,
        'offset': offset
    }
    response = requests.get(url, headers=headers, params=params)
    return response.json()

# Fetch and store all locations
offset = 1
limit = 1000
while True:
    data = fetch_locations(offset, limit)
    locations = data.get('results', [])
    if not locations:
        break

    # Insert locations into the database
    for location in locations:
        cursor.execute('''
        INSERT OR IGNORE INTO locations (id, name, mindate, maxdate, datacoverage)
        VALUES (?, ?, ?, ?, ?)
        ''', (
            location['id'],
            location['name'],
            location['mindate'],
            location['maxdate'],
            location['datacoverage']
        ))

    offset += limit
    print(f"Fetched {offset} locations")

# Commit and close the database connection
conn.commit()
conn.close()

In [3]:
# Add new columns city and alpha_2
conn.execute('ALTER TABLE locations ADD COLUMN city TEXT')
conn.execute('ALTER TABLE locations ADD COLUMN alpha_2 TEXT')

# Update the table by splitting the name column
conn.execute('''
UPDATE locations
SET city = SUBSTR(name, 1, INSTR(name, ',') - 1),
    alpha_2 = SUBSTR(id, INSTR(id, ':') + 1, 2)
''')

# Commit the changes
conn.commit()

In [5]:
pd.read_sql_query('''
SELECT *
from locations
where city = 'Barcelona'
limit 3
''', conn)

,id,name,mindate,maxdate,datacoverage,city,alpha_2
0,CITY:SP000001,"Barcelona, SP",1913-08-01,2024-09-24,1.0,Barcelona,SP
1,CITY:VE000001,"Barcelona, VE",1968-01-01,2024-09-24,1.0,Barcelona,VE
